# Data Cleaning – Retail Store Sales

Purpose of this notebook:
- Inspect data quality issues
- Apply deterministic cleaning rules
- Produce a clean, analysis-ready dataset


In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [4]:
data_path = "../data/raw/retail_store_sales_dirty.csv"
df = pd.read_csv(data_path)

df.head()


,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False


In [9]:
df.shape

(12575, 11)

In [12]:
df.columns.tolist()

['Transaction ID',
 'Customer ID',
 'Category',
 'Item',
 'Price Per Unit',
 'Quantity',
 'Total Spent',
 'Payment Method',
 'Location',
 'Transaction Date',
 'Discount Applied']

## 1. Data Profiling

Initial inspection of dataset structure, data types, and missing values.
The goal is to understand data quality issues before applying any transformations.


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12575 entries, 0 to 12574
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    12575 non-null  object 
 1   Customer ID       12575 non-null  object 
 2   Category          12575 non-null  object 
 3   Item              11362 non-null  object 
 4   Price Per Unit    11966 non-null  float64
 5   Quantity          11971 non-null  float64
 6   Total Spent       11971 non-null  float64
 7   Payment Method    12575 non-null  object 
 8   Location          12575 non-null  object 
 9   Transaction Date  12575 non-null  object 
 10  Discount Applied  8376 non-null   object 
dtypes: float64(3), object(8)
memory usage: 1.1+ MB


In [14]:
df.describe()

,Price Per Unit,Quantity,Total Spent
count,11966.000000,11971.000000,11971.000000
mean,23.365912,5.536380,129.652577
std,10.743519,2.857883,94.750697
min,5.000000,1.000000,5.000000
25%,14.000000,3.000000,51.000000
50%,23.000000,6.000000,108.500000
75%,33.500000,8.000000,192.000000
max,41.000000,10.000000,410.000000


In [15]:
df['Payment Method'].value_counts(dropna=False)

Payment Method
Cash              4310
Digital Wallet    4144
Credit Card       4121
Name: count, dtype: int64

In [16]:
df['Category'].value_counts(dropna=False)

Category
Furniture                             1591
Electric household essentials         1591
Food                                  1588
Milk Products                         1584
Butchers                              1568
Beverages                             1567
Computers and electric accessories    1558
Patisserie                            1528
Name: count, dtype: int64

In [25]:
df['Item'].value_counts(dropna=False).head(20)

Item
NaN             1213
Item_2_BEV       126
Item_25_FUR      113
Item_11_FUR      110
Item_1_MILK      109
Item_16_MILK     109
Item_5_FUR       107
Item_20_BUT      107
Item_19_CEA      106
Item_14_FOOD     106
Item_11_MILK     106
Item_24_FUR      105
Item_8_EHE       105
Item_19_MILK     105
Item_13_FOOD     101
Item_12_CEA      101
Item_14_BEV      100
Item_22_BUT      100
Item_12_PAT      100
Item_5_CEA        98
Name: count, dtype: int64

In [20]:
df[df['Item'].isna()][['Category', 'Price Per Unit', 'Quantity', 'Total Spent']].head(10)

,Category,Price Per Unit,Quantity,Total Spent
5,Patisserie,NaN,10.0,200.0
7,Furniture,33.5,NaN,NaN
11,Milk Products,NaN,8.0,52.0
15,Beverages,24.5,NaN,NaN
17,Milk Products,NaN,10.0,275.0
19,Furniture,35.0,NaN,NaN
21,Milk Products,NaN,3.0,105.0
25,Furniture,39.5,NaN,NaN
32,Food,NaN,8.0,196.0
34,Patisserie,23.0,NaN,NaN


In [21]:
df[['Item', 'Price Per Unit', 'Quantity', 'Total Spent']].isna().sum()

Item              1213
Price Per Unit     609
Quantity           604
Total Spent        604
dtype: int64

### Key Data Quality Issues Identified

- Missing values in critical transaction fields (Item, Price Per Unit, Quantity, Total Spent)
- Missing values in Discount Applied
- Incorrect data types for Transaction Date and Discount Applied
- Potential inconsistency in Total Spent calculation
- Partially filled (incomplete) transactions


## 2. Cleaning Plan

Cleaning rules were defined before modifying the data.
Only deterministic and explainable transformations were allowed.


### Rules Summary

- Standardize data types without changing meaning
- Validate transaction consistency before fixing values
- Restore numeric values only when exactly one value is missing
- Remove transactions with two or more missing numeric fields
- Restore Item only when price uniquely identifies a product
- Ambiguous Item values are labeled as "Unknown"


In [26]:
df['Transaction Date'].head(10)

0    2024-04-08
1    2023-07-23
2    2022-10-05
3    2022-05-07
4    2022-10-02
5    2023-11-30
6    2023-06-10
7    2024-04-02
8    2023-04-26
9    2024-03-14
Name: Transaction Date, dtype: object

## 3. Data Cleaning Execution


### 3.1 Data Type Standardization

Transaction Date and Discount Applied were converted to appropriate data types.
No values were modified at this stage.


In [28]:
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], errors='coerce')

In [29]:
df['Transaction Date'].dtype, df['Transaction Date'].isna().sum()

(dtype('<M8[ns]'), np.int64(0))

In [31]:
df['Discount Applied'].value_counts(dropna=False)

Discount Applied
True     4219
NaN      4199
False    4157
Name: count, dtype: int64

In [34]:
df['Discount Applied'] = (df['Discount Applied'].fillna(False).astype(bool))

In [35]:
df['Discount Applied'].dtype, df['Discount Applied'].value_counts()

(dtype('bool'),
 Discount Applied
 False    8356
 True     4219
 Name: count, dtype: int64)

Missing values in Discount Applied were interpreted as no discount applied.
This assumption is based on the binary nature of the field and typical transactional system behavior, where discounts are explicitly recorded when present.

In [37]:
mask_complete = df[['Price Per Unit', 'Quantity', 'Total Spent']].notna().all(axis=1)
mask_complete.sum()

np.int64(11362)

### 3.2 Transaction Consistency Check

The following rule was validated for all complete transactions:

Total Spent = Price Per Unit × Quantity


In [38]:
df.loc[mask_complete, 'expected_total'] = (
    df.loc[mask_complete, 'Price Per Unit'] *
    df.loc[mask_complete, 'Quantity']
)

In [39]:
df.loc[mask_complete, 'diff'] = (
    df.loc[mask_complete, 'Total Spent'] -
    df.loc[mask_complete, 'expected_total']
)

In [40]:
(df.loc[mask_complete, 'diff'] != 0).sum()

np.int64(0)

Result:
- All validated transactions are consistent
- No recalculation was required


In [41]:
num_cols = ['Price Per Unit', 'Quantity', 'Total Spent']
num_missing_count = df[num_cols].isna().sum(axis=1)

num_missing_count.value_counts().sort_index()

0    11362
1      609
2      604
Name: count, dtype: int64

### 3.3 Handling Missing Numeric Values

Approach:
- Exactly one missing numeric field -> restored deterministically
- Two or more missing numeric fields -> transaction removed


In [46]:
#copy for cleaning
df_work = df.copy()

#masks
m1 = (num_missing_count == 1)
has_price = df_work['Price Per Unit'].notna()
has_qty = df_work['Quantity'].notna()
has_total = df_work['Total Spent'].notna()

#restore Total Spent
mask_total = m1 & has_price & has_qty & df_work['Total Spent'].isna()
df_work.loc[mask_total, 'Total Spent'] = (
    df_work.loc[mask_total, 'Price Per Unit'] *
    df_work.loc[mask_total, 'Quantity']
)

#restore Price Per Unit
mask_price = m1 & has_qty & has_total & df_work['Price Per Unit'].isna()
df_work.loc[mask_price, 'Price Per Unit'] = (
    df_work.loc[mask_price, 'Total Spent'] /
    df_work.loc[mask_price, 'Quantity']
)

#restore Quantity
mask_qty = m1 & has_price & has_total & df_work['Quantity'].isna()
df_work.loc[mask_qty, 'Quantity'] = (
    df_work.loc[mask_qty, 'Total Spent'] /
    df_work.loc[mask_qty, 'Price Per Unit']
)

In [47]:
df_work[num_cols].isna().sum()

Price Per Unit      0
Quantity          604
Total Spent       604
dtype: int64

In [48]:
df_work = df_work[num_missing_count < 2]

In [49]:
df_work[num_cols].isna().sum()

Price Per Unit    0
Quantity          0
Total Spent       0
dtype: int64

Result:
- 609 transactions restored
- 604 incomplete transactions removed


### 3.4 Item Recovery

Item values were restored using a price-based rule:
- Unique price → Item restored
- Non-unique or ambiguous price → Item set to "Unknown"


In [50]:
price_item_map = (
    df_work
    .dropna(subset=['Item'])
    .groupby('Price Per Unit')['Item']
    .nunique()
)

In [51]:
unique_price_to_item = (
    df_work
    .dropna(subset=['Item'])
    .groupby('Price Per Unit')['Item']
    .agg(lambda x: x.iloc[0])
)

In [52]:
mask_item_na = df_work['Item'].isna()

df_work.loc[
    mask_item_na & df_work['Price Per Unit'].isin(unique_price_to_item.index),
    'Item'
    ] = df_work.loc[
        mask_item_na & df_work['Price Per Unit'].isin(unique_price_to_item.index),
        'Price Per Unit'
        ].map(unique_price_to_item)

In [53]:
df_work['Item'] = df_work['Item'].fillna('Unknown')

In [56]:
df_work['Item'].isna().sum(), df_work['Item'].value_counts().head()

(np.int64(0),
 Item
 Item_25_FUR     147
 Item_20_BUT     138
 Item_19_MILK    135
 Item_2_BEV      126
 Item_22_BUT     125
 Name: count, dtype: int64)

### 3.5 Duplicates and Integrity Checks

Transaction IDs were validated for uniqueness.
Duplicate rows were checked and not found.


In [58]:
df_work['Transaction ID'].duplicated().sum()

np.int64(0)

In [59]:
df_work.duplicated().sum()

np.int64(0)

In [60]:
df_work.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11971 entries, 0 to 12574
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    11971 non-null  object        
 1   Customer ID       11971 non-null  object        
 2   Category          11971 non-null  object        
 3   Item              11971 non-null  object        
 4   Price Per Unit    11971 non-null  float64       
 5   Quantity          11971 non-null  float64       
 6   Total Spent       11971 non-null  float64       
 7   Payment Method    11971 non-null  object        
 8   Location          11971 non-null  object        
 9   Transaction Date  11971 non-null  datetime64[ns]
 10  Discount Applied  11971 non-null  bool          
 11  expected_total    11362 non-null  float64       
 12  diff              11362 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(5), object(6)
memory usage: 1.2+ MB


In [61]:
df_work.describe()

,Price Per Unit,Quantity,Total Spent,Transaction Date,expected_total,diff
count,11971.000000,11971.000000,11971.000000,11971,11362.000000,11362.0
mean,23.360872,5.536380,129.652577,2023-07-13 02:26:23.627099136,129.642537,0.0
min,5.000000,1.000000,5.000000,2022-01-01 00:00:00,5.000000,0.0
25%,14.000000,3.000000,51.000000,2022-09-29 00:00:00,52.000000,0.0
50%,23.000000,6.000000,108.500000,2023-07-13 00:00:00,108.500000,0.0
75%,33.500000,8.000000,192.000000,2024-04-24 00:00:00,192.000000,0.0
max,41.000000,10.000000,410.000000,2025-01-18 00:00:00,410.000000,0.0
std,10.741889,2.857883,94.750697,NaN,94.662368,0.0


In [62]:
df_work.isna().sum()

Transaction ID        0
Customer ID           0
Category              0
Item                  0
Price Per Unit        0
Quantity              0
Total Spent           0
Payment Method        0
Location              0
Transaction Date      0
Discount Applied      0
expected_total      609
diff                609
dtype: int64

In [64]:
df_work.shape

(11971, 13)

In [65]:
df_final = df_work.drop(columns=['expected_total', 'diff'])

In [68]:
df_final.shape

(11971, 11)

In [69]:
df_final.isna().sum()

Transaction ID      0
Customer ID         0
Category            0
Item                0
Price Per Unit      0
Quantity            0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
Discount Applied    0
dtype: int64

In [70]:
output_path = "../data/cleaned/retail_store_sales_clean.csv"
df_final.to_csv(output_path, index=False)

## 4. Final Validation

- No missing values remaining
- All numeric fields are consistent
- Dataset is ready for analysis


Final dataset shape: 11,971 rows × 11 columns
